In [12]:
import preprocessing as preproc
import helpers
from tqdm import tqdm
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten, GlobalMaxPooling1D
from keras.optimizers import Adam, SGD
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
start_time = time.time()

full_dataset = False

train_data, y, test_data = helpers.get_processed_data(full_dataset=full_dataset)

In [13]:
vectorizer = CountVectorizer()
count_vect = vectorizer.fit_transform(train_data)

In [4]:
vocabulary = vectorizer.get_feature_names()
no_vocabulary = len(vocabulary)
print('num of words: ', no_vocabulary)

num of words:  75341


In [5]:
num_words = 60000
sum_words = X_train.sum(axis=0)
word_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
word_freq_sorted = sorted(word_freq, key = lambda x : x[1], reverse=True)
word_sorted = [x[0] for x in word_freq_sorted]
print('first 5: ', word_sorted[:5])

first 5:  ['you', 'the', 'to', 'negative', 'number']


In [17]:
type(train_data)

list

In [6]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_data)
sequences = tokenizer.texts_to_sequences(train_data)

In [7]:
length = []
for x in train_data:
    length.append(len(x.split()))
print('max sentence length:', max(length))
tweet_max_length = 80

max sentence length: 74


In [8]:
train_data_seq = pad_sequences(sequences, maxlen=tweet_max_length)

In [ ]:
# sequences_val = tokenizer.texts_to_sequences(x_validation)
# x_val_seq = pad_sequences(sequences_val, maxlen=45)

In [ ]:
# num_words = 100000
# embedding_matrix = np.zeros((num_words, 200))
# for word, i in tokenizer.word_index.items():
#     if i >= num_words:
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [11]:
model_cnn_02 = Sequential()
e = Embedding(num_words, 200, input_length=tweet_max_length)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=0.15, random_state=42)

In [16]:
model_cnn_02.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=1)

Train on 170000 samples, validate on 30000 samples
Epoch 1/5
170000/170000 [==============================] - 666s 4ms/step - loss: 0.4099 - acc: 0.8058 - val_loss: 0.3781 - val_acc: 0.8253
Epoch 2/5
103200/170000 [=================>............] - ETA: 4:36 - loss: 0.2979 - acc: 0.8695

KeyboardInterrupt: 